# HSE Summer School NLP&DA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [26]:
def loadAnswer(number):
    with open("Texts/art{}.opin.txt".format(number)) as f:
        d = f.read()
    return d

def loadText(number):
    with open("Texts/art{}.txt".format(number)) as f:
        d = f.read()
    return d



In [29]:
with open("Texts/art2.ann") as f:
    print(f.read())

T1	PER 1 7	Author
T2	PER 9 16	Unknown
T3	PER 22 28	Author
T4	PER 30 37	Unknown
T5	LOC 39 45	Россия
T6	LOC 72 77	Сирии
T7	PER 81 87	Author
T8	PER 89 96	Unknown
T9	PER 147 153	Author
T10	PER 155 162	Unknown
T11	LOC 164 170	Россия
T12	LOC 195 200	Сирии
T13	PER 249 255	Author
T14	PER 257 264	Unknown
T15	PER 299 305	Author
T16	PER 307 314	Unknown
T17	PER 491 497	Author
T18	PER 499 506	Unknown
T19	LOC 521 527	Москвы
T20	LOC 540 545	Сирии
T21	LOC 584 590	России
T22	PER 711 717	Author
T23	PER 719 726	Unknown
T24	LOC 728 732	Иран
T25	PER 773 785	Башара Асада
T26	ORG 815 821	Кремля
T27	PER 825 831	Author
T28	PER 833 840	Unknown
T29	LOC 842 848	Россия
T30	LOC 958 963	Ирана
T31	PER 986 992	Author
T32	PER 994 1001	Unknown
T33	LOC 1036 1042	России
T34	LOC 1096 1103	Дамаске
T35	PER 1236 1242	Author
T36	PER 1244 1251	Unknown
T37	LOC 1266 1269	США
T38	PER 1288 1300	Барака Обамы
T39	LOC 1408 1413	Сирии
T40	PER 1417 1423	Author
T41	PER 1425 1432	Unknown
T42	LOC 1491 1497	Россию
T43	PER 1553 1559	Author
T

In [27]:
print(loadText(4))

{Author, Unknown} Почему лидеры Востока едут к Путину, а не к Обаме

{Author, Unknown} inosmi.ruПосмотреть оригиналавгуст 5-го, 2016

{Author, Unknown} Россия менее сильна в военном отношении, но она в гораздо большей степени готова действовать, и это изменило динамику на Ближнем Востоке


{Author, Unknown} В настоящий момент военный потенциал США на Ближнем Востоке в значительной мере превосходит военный потенциал России: в этом регионе сейчас находится 35 тысяч американских военнослужащих, сотни американских самолетов и всего 2 тысячи российских военных и около 50 российских самолетов. 
{Author, Unknown} Тем не менее, лидеры ближневосточных государств совершают поездки в Москву, чтобы встретиться там с Владимиром Путиным, а вовсе не в Вашингтон. 
{Author, Unknown} Две недели назад в Москву приехал премьер-министр Израиля Биньямин Нетаньяху (Benjamin Netanyahu) — это уже вторая его поездка в Россию с осени прошлого года — и скоро король Саудовской Аравии Салман тоже посетит Москву. 
{

In [28]:
print(loadAnswer(4))

Израиль, Россия, pos, current
Саудовская Аравия, Россия, pos, current
Египет, Россия, pos, current
Обама, Путин, neg, current
Израиль, США, neg, current
Саудовская Аравия, США, neg, current
Египет, США, neg, current
author, Россия, pos, current
author, Обама, neg, current
author, Путин, neg, current
Путин, Асад, pos, current
Саудовская Аравия, Иран, neg, current
Иран, Сирия, pos, current
Обама, Асад, neg, current
Государства Персидского Залива, Иран, neg, current
Саудовская Аравия, Йемен, neg, current
Йемен, Саудовская Аравия, neg, current
США, Иран, neg, current
Обама, Иран, neg, current
author, Иран, neg, current
Обама, ИГИЛ, neg, current
США, ИГИЛ, neg, current
author, Асад, neg, current


